<center><h2>ALTeGraD 2022<br>Lab Session 3: NLP Frameworks</h2> 15 / 11 / 2022<br> M. Kamal Eddine, H. Abdine<br><br>


<b>Student name:</b> Sicheng Mao

</center>

In this lab you will learn how to use Fairseq and HuggingFace transformers - The most used libraries by researchers and developers to pretrain and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative.

In the first part of this lab, you will finetune the given model on model on CLS_Books dataset using <b>Fairseq</b> by following these steps:<br>

 1- <b>Tokenize the reviews</b> (Train, Valid and Test) using trained sentencepiece tokenizer provided alongside the pretrained model.[using sentencepiece library and setting the parameter <b>out_type=str</b> in the encode function].<br>
 2- <b>Binarize the tokenized reviews and their labels</b> using the preprocess python script provided in Fairseq.<br>
 3- <b>Fintune the pretrained $RoBERTa_{small}^{fr}$ model</b> using the train python script provided in Fairseq.<br>
 
 Finally, you will finish the first part by training a random $RoBERTa_{small}^{fr}$ model on the CLS_Books dataset and compare the results against the pretrained model while <b>visualizing the accuracies on tensorboard</b>.

 In the second part of this lab, you will use <b>HuggingFace's transformers</b> library to perform the finrtuning done previously with Fairseq.



# <b>Part 1: Fairseq</b>

## <b>Preparing the environment and installing libraries, model and data</b>

In this section, we will setup the environment on Google Colab (first cell), download the pretraind model (second cell) and the finetuning dataset (third cell). In case you are using your personal computer maket sure to:

1- Use Ubuntu (or any similar linux distribution) or MacOS. <b> P.S. In case you have Windows, please use Google Colab. We won't respond to any question regarding errors on Windows. </b>

2- <b>Use Anaconda</b> and create new environment if you already installed Fairseq since we will be using a slightly modified version of this library.

3- <b>Do not run the following three cells</b>. Instead, use their content on your personal command line.

In [ ]:
!mkdir altegrad.lab3 && cd altegrad.lab3 && mkdir libs 
%cd altegrad.lab3/libs
!git clone https://github.com/hadi-abdine/fairseq
!pip install git+https://github.com/hadi-abdine/fairseq
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tensorboardX
%load_ext tensorboard

/content/altegrad.lab3/libs
Cloning into 'fairseq'...
remote: Enumerating objects: 19737, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 19737 (delta 0), reused 4 (delta 0), pack-reused 19730
Receiving objects: 100% (19737/19737), 18.52 MiB | 22.58 MiB/s, done.
Resolving deltas: 100% (14337/14337), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/hadi-abdine/fairseq to /tmp/pip-req-build-plxcnykp
  Running command git clone -q https://github.com/hadi-abdine/fairseq /tmp/pip-req-build-plxcnykp
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████|

In [ ]:
!cd .. && mkdir models 
%cd ../models
!wget -c "https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%21267604&authkey=ANaaKIDigQPyJlM" -O "model_fairseq.zip"
!unzip model_fairseq.zip
!rm model_fairseq.zip
!rm -rf __MACOSX/

/content/altegrad.lab3/models
--2022-11-19 08:35:21--  https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%21267604&authkey=ANaaKIDigQPyJlM
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://kiebga.am.files.1drv.com/y4malmMYra-lF9upyMRpfHsc2ZuMglBMciBn3sSAxHdcYi2YFE2c99h5HCl_NCkSpDkccDRfndjnIaXrU86JZ_6QmTvTo72FSn4jdxkvEJuHeGKL6ixW50R8NmcOcm-S2NvKMQavaQePoqcgYBAxEb3fn-EzY-uU6CuCojaMLwXlTSQqWr_KrO3uqtwiaIR6kTmV23MnjHXgHXSxs_J9-0Qsg/RoBERTa_small_fr.zip?download&psid=1 [following]
--2022-11-19 08:35:22--  https://kiebga.am.files.1drv.com/y4malmMYra-lF9upyMRpfHsc2ZuMglBMciBn3sSAxHdcYi2YFE2c99h5HCl_NCkSpDkccDRfndjnIaXrU86JZ_6QmTvTo72FSn4jdxkvEJuHeGKL6ixW50R8NmcOcm-S2NvKMQavaQePoqcgYBAxEb3fn-EzY-uU6CuCojaMLwXlTSQqWr_KrO3uqtwiaIR6kTmV23MnjHXgHXSxs_J9-0Qsg/RoBERTa_small_fr.zip?download&psid=1
Resolving ki

In [ ]:
!cd .. && mkdir data
%cd ../data
!wget -c "https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%21267606&authkey=AA8Td6LoeijplD4" -O "cls.books.zip"
!unzip cls.books.zip
!rm cls.books.zip
!rm -rf __MACOSX/
%cd ..

/content/altegrad.lab3/data
--2022-11-19 08:35:44--  https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%21267606&authkey=AA8Td6LoeijplD4
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yapqxq.am.files.1drv.com/y4mLVYhbQYCVWPyDq0Wy4qLFrsUx6Tk-AKlEadWVO7ywUfXrh78Hn0ZxkzlgdN2rzchbRTKcYpUSkU4BpSgS3WIO8GZVpeYPErBSGRXRa3T7Sv3BYTL2_op6TljUFBqHGISaPAkCEYetUphmb-Gz-MEBzfRBMV1FAGqMsuPAan5VBkQY7Laep8cie7UwSc0PMk1APF2Auc4npKuCXjtpPkcnQ/cls.books.zip?download&psid=1 [following]
--2022-11-19 08:35:45--  https://yapqxq.am.files.1drv.com/y4mLVYhbQYCVWPyDq0Wy4qLFrsUx6Tk-AKlEadWVO7ywUfXrh78Hn0ZxkzlgdN2rzchbRTKcYpUSkU4BpSgS3WIO8GZVpeYPErBSGRXRa3T7Sv3BYTL2_op6TljUFBqHGISaPAkCEYetUphmb-Gz-MEBzfRBMV1FAGqMsuPAan5VBkQY7Laep8cie7UwSc0PMk1APF2Auc4npKuCXjtpPkcnQ/cls.books.zip?download&psid=1
Resolving yapqxq.am.files.1d

## <b> Number of parameters of the model</b>

In this section you have to compute the number of parameters of $RoBERTa_{small}^{fr}$ using PyTorch. (<b>Hint:</b> you can check the architecture of the model using model['model'])

In [ ]:
import torch

model = torch.load("models/RoBERTa_small_fr/model.pt")

0


In [ ]:
type(model)

dict

In [ ]:
# your code here
type(model['model'])

collections.OrderedDict

In [ ]:
for (key, value) in model['model'].items():
  print(key, value.shape)

encoder.sentence_encoder.version torch.Size([1])
encoder.sentence_encoder.embed_tokens.weight torch.Size([32000, 512])
encoder.sentence_encoder.embed_positions.weight torch.Size([258, 512])
encoder.sentence_encoder.layernorm_embedding.weight torch.Size([512])
encoder.sentence_encoder.layernorm_embedding.bias torch.Size([512])
encoder.sentence_encoder.layers.0.self_attn.k_proj.weight torch.Size([512, 512])
encoder.sentence_encoder.layers.0.self_attn.k_proj.bias torch.Size([512])
encoder.sentence_encoder.layers.0.self_attn.v_proj.weight torch.Size([512, 512])
encoder.sentence_encoder.layers.0.self_attn.v_proj.bias torch.Size([512])
encoder.sentence_encoder.layers.0.self_attn.q_proj.weight torch.Size([512, 512])
encoder.sentence_encoder.layers.0.self_attn.q_proj.bias torch.Size([512])
encoder.sentence_encoder.layers.0.self_attn.out_proj.weight torch.Size([512, 512])
encoder.sentence_encoder.layers.0.self_attn.out_proj.bias torch.Size([512])
encoder.sentence_encoder.layers.0.self_attn_laye

In [ ]:
n_parameters = 0
layer_include = ['embed','layers']
layer_exclude = ['bias', 'norm']
for (key, value) in model['model'].items():
  if any(word in key for word in layer_include) and not any(word in key for word in layer_exclude):
    print(key, value.numel())
    n_parameters += value.numel()
print(n_parameters)

encoder.sentence_encoder.embed_tokens.weight 16384000
encoder.sentence_encoder.embed_positions.weight 132096
encoder.sentence_encoder.layers.0.self_attn.k_proj.weight 262144
encoder.sentence_encoder.layers.0.self_attn.v_proj.weight 262144
encoder.sentence_encoder.layers.0.self_attn.q_proj.weight 262144
encoder.sentence_encoder.layers.0.self_attn.out_proj.weight 262144
encoder.sentence_encoder.layers.0.fc1.weight 262144
encoder.sentence_encoder.layers.0.fc2.weight 262144
encoder.sentence_encoder.layers.1.self_attn.k_proj.weight 262144
encoder.sentence_encoder.layers.1.self_attn.v_proj.weight 262144
encoder.sentence_encoder.layers.1.self_attn.q_proj.weight 262144
encoder.sentence_encoder.layers.1.self_attn.out_proj.weight 262144
encoder.sentence_encoder.layers.1.fc1.weight 262144
encoder.sentence_encoder.layers.1.fc2.weight 262144
encoder.sentence_encoder.layers.2.self_attn.k_proj.weight 262144
encoder.sentence_encoder.layers.2.self_attn.v_proj.weight 262144
encoder.sentence_encoder.laye

## <b>Tokenizing the reviews</b>

In this section we will tokenize the finetuning dataset using sentenpiece tokenizer. We have three splits in our datase: train valid and test sets. 

In this task you have to use the trained sentencepiece tokenizer (RoBERTa_small_fr/sentencepiece.bpe.model) to tokenize the three files <b>train.review</b>, <b>valid.review</b> and <b>test.review</b> and output the three files <b>train.spm.review</b>, <b>valid.spm.review</b> and <b>test.spm.review</b> containing the tokenized reviews.

In [ ]:
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='models/RoBERTa_small_fr/sentencepiece.bpe.model')

In [ ]:
SPLITS=['train', 'test', 'valid']
SENTS="review"

for split in SPLITS:
    with open('data/cls.books/'+split+'.'+SENTS, 'r') as f:
        reviews = f.readlines()
        print(reviews[0])
        reviews = [' '.join(s.encode(review, out_type = str)) for review in reviews]# tokenize data
        
        #It should look something like that
        #▁An ci enne ▁VS ▁Nouvelle ▁version ▁plus
        print(reviews[0]) 
    with open('data/cls.books/'+split+'.spm.'+SENTS, 'w') as f:
        for review in reviews:
          f.write(review+'\n')


Ce livre est tout simplement magique !  il vaut le detour suspense, humour, magie, tristesse, courage,sont parfaitement regroupés dans cet ouvrage ... Un vrai chef-d'oeuvre ! Bien que le sort de Harry soit prévisible, en raison de sa sortie quasi simultanée avec les deux tomes suivants ; ce livre est un régal que se soit pour les petits ou les grands !!! Johanne Kathleen Rowling a su alier plusieurs histoires qui paraissent totalement différentes mais avec une fin qui leur est communne et quasi introuvable jusqu'à la fin ! Elle à également su comment faire pour attirer une génération de non lecteurs avec ses livres "énormes" et sans images ! Entre nous il ya de quoi se demander qui le magicien Harry ou elle ? Bravo !!!

▁Ce ▁livre ▁est ▁tout ▁simplement ▁mag ique ▁! ▁il ▁vaut ▁le ▁de tour ▁suspens e , ▁ humour , ▁magie , ▁triste sse , ▁courage , son t ▁parfaitement ▁re group és ▁dans ▁cet ▁ ouvrage ▁... ▁Un ▁vrai ▁chef - d ' oeuvre ▁! ▁Bien ▁que ▁le ▁sort ▁de ▁Harry ▁soit ▁pré visible 

## <b>Binarizing the finetuning dataset</b>

In this section, you have to binarize the CLS_Books dataset using the <b>fairseq/fairseq_cli/preprocess.py</b> script:

1- Binarize the tokenized reviews and put the output in <b>data/cls-books-bin/input0</b>. Note: Our pretrained model's embedding matrix contains only the embedding of the vocab listed in the dictionary <b>dict.txt</b>

2- Binarize the labels (train.label, valid.label and test.label files) and put the output in <b>data/cls-books-bin/label</b>.

Use `!python libs/fairseq/fairseq_cli/preprocess.py --help` to get details about the arguments and visit the fairseq github repository for further help.

In [ ]:
!python libs/fairseq/fairseq_cli/preprocess.py --help

usage: preprocess.py [-h] [--no-progress-bar] [--log-interval LOG_INTERVAL]
                     [--log-format {json,none,simple,tqdm}]
                     [--log-file LOG_FILE] [--aim-repo AIM_REPO]
                     [--aim-run-hash AIM_RUN_HASH]
                     [--tensorboard-logdir TENSORBOARD_LOGDIR]
                     [--wandb-project WANDB_PROJECT] [--azureml-logging]
                     [--seed SEED] [--cpu] [--tpu] [--bf16]
                     [--memory-efficient-bf16] [--fp16]
                     [--memory-efficient-fp16] [--fp16-no-flatten-grads]
                     [--fp16-init-scale FP16_INIT_SCALE]
                     [--fp16-scale-window FP16_SCALE_WINDOW]
                     [--fp16-scale-tolerance FP16_SCALE_TOLERANCE]
                     [--on-cpu-convert-precision]
                     [--min-loss-scale MIN_LOSS_SCALE]
                     [--threshold-loss-scale THRESHOLD_LOSS_SCALE] [--amp]
                     [--amp-batch-retries AMP_BATCH_RETRIE

In [ ]:
TEXT = 'data/cls.books'
input_dir = 'data/cls-books-bin/input0'
label_dir = 'data/cls-books-bin/label'

In [ ]:
!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \
              --source-lang fr \
              --trainpref $TEXT/train.spm.review \
              --validpref $TEXT/valid.spm.review \
              --testpref $TEXT/test.spm.review \
              --srcdict models/RoBERTa_small_fr/dict.txt \
              --dict-only \
              --destdir $input_dir \
              --workers 8)
              #fill me - binarize the tokenized reviews

2022-11-19 10:18:42 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/cls-books-bin/input0', dict_only=True, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='fr', src

In [ ]:
!(python libs/fairseq/fairseq_cli/preprocess.py \
              --only-source \
              --source-lang fr \
              --trainpref $TEXT/train.label \
              --validpref $TEXT/valid.label \
              --testpref $TEXT/test.label \
              --srcdict models/RoBERTa_small_fr/dict.txt \
              --dict-only\
              --destdir $label_dir \
              --workers 8)#fill me - binarize the labels

2022-11-19 10:19:39 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/cls-books-bin/label', dict_only=True, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='fr', srcd

## <b>Finetuning $RoBERTa_{small}^{fr}$</b>

In this section you will use <b>fairseq/fairseq_cli/train.py</b> python script to finetune the pretrained model on the CLS_Books dataset (binarized data) for three different seeds: 0, 1 and 2. 

Make sure to use the following hyper-parameters: $\textit{batch size}=8, \textit{max number of epochs}: 5, \textit{optimizer}: Adam, \textit{max learning rate}: 1e-05,  \textit{warm up ratio}: 0.06, \textit{learning rate scheduler}: linear$

In [ ]:
DATA_SET='books'
TASK= # fill me, sentence prediction task on fairseq
MODEL='RoBERTa_small_fr'
DATA_PATH= # fill me 
MODEL_PATH= # fill me
MAX_EPOCH= # fill me 
MAX_SENTENCES= # fill me, batch size
MAX_UPDATE= # fill me, n_epochs * n_train_examples / total batch size 
LR= # fill me
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score. 
METRIC = # fill me, use the accuracy metric
NUM_CLASSES=#fill me, number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = # fill me, warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric $METRIC \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $WARMUP \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)


## <b>Random $RoBERTa_{small}^{fr}$ model training:</b>

In this section you have to finetune a random checkpinf of the model $RoBERTa_{small}^{fr}$ using the same setting as before (<b>Hint:</b> an unexisted model path will not give you an error) 

In [ ]:
DATA_SET='books'
TASK= # fill me, sentence prediction task on fairseq
MODEL='RoBERTa_small_fr_random'
DATA_PATH= # fill me 
MODEL_PATH= # fill me
MAX_EPOCH= # fill me 
MAX_SENTENCES= # fill me, batch size
MAX_UPDATE= # fill me, n_epochs * n_train_examples / total batch size 
LR= # fill me
VALID_SUBSET='valid,test' # for simplicity we will validate on both valid and test set, and then pick the value of test set corresponding the best validation score. 
METRIC = # fill me, use the accuracy metric
NUM_CLASSES=#fill me, number of classes
SEEDS=3
CUDA_VISIBLE_DEVICES=0
WARMUP = # fill me, warmup ratio=6% of the whole training

In [ ]:
for SEED in range(SEEDS):
  TENSORBOARD_LOGS= 'tensorboard_logs/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_mu'+str(MAX_UPDATE)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/fairseq/fairseq_cli/train.py $DATA_PATH \
                --restore-file $MODEL_PATH \
                --batch-size $MAX_SENTENCES \
                --task $TASK \
                --update-freq 1 \
                --seed $SEED \
                --reset-optimizer --reset-dataloader --reset-meters \
                --init-token 0 \
                --separator-token 2 \
                --arch roberta_small \
                --criterion sentence_prediction \
                --num-classes $NUM_CLASSES \
                --weight-decay 0.01 \
                --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-08 \
                --maximize-best-checkpoint-metric \
                --best-checkpoint-metric $METRIC \
                --save-dir $SAVE_DIR \
                --lr-scheduler polynomial_decay \
                --lr $LR \
                --max-update $MAX_UPDATE \
                --total-num-update $MAX_UPDATE \
                --no-epoch-checkpoints \
                --no-last-checkpoints \
                --tensorboard-logdir $TENSORBOARD_LOGS \
                --log-interval 5 \
                --warmup-updates $WARMUP \
                --max-epoch $MAX_EPOCH \
                --keep-best-checkpoints 1 \
                --max-positions 256 \
                --valid-subset $VALID_SUBSET \
                --shorten-method 'truncate' \
                --no-save \
                --distributed-world-size 1)

## <b>Tensorboard Visualisation </B>

In the this we will use tensorboard to visualize the training, validation and test accuracies. <b>Include and analyse in you report a screenshot of the test accuracy of the six models</b>.

In [ ]:
%tensorboard --logdir tensorboard_logs

# <b>Part 2: HuggingFace's Transfromers</b>

In this part of the lab, we will finetune a HuggingFace checkpoint of our $RoBERTa_{small}^{fr}$ on the CLS_Books dataset. Like in the first part we will start by downloading the HuggingFace checkpoint and <b>preparing a json format of the CLS_Books dataset</b> (Which is suitable for HuggingFace's checkpoints finetuning). Again, if you are using you personal computer, do not run the following cell and use its content to download the files on you computer, since - depending on you operating system - running this cell will produce errors. 

In [ ]:
%cd models
!wget -c "https://onedrive.live.com/download?cid=AE69638675180117&resid=AE69638675180117%21267607&authkey=APJub1wVzVLAoR8" -O "model_huggingface.zip"
!unzip model_huggingface.zip
!rm model_huggingface.zip
!rm -rf __MACOSX/

%cd ../data
!mkdir cls.books-json

%cd ..

## <b>Converting the CLS_Books dataset to json line files</b>

Unlike Fairseq, you do not need to perform tokenization and binarization in Hugging Face transformer library. However, in order to use the implemented script in the transformers library, you need to convert your data to json line files (for each split: train, valid and test)

for instance, each line inside you file will consist of one and one sample only, contaning the review (accessed by the key <i>sentence1</i> and its label, accessed by the key <i>label</i>. Below you can find an example from <i>valid.json</i> file.

Note that these instructions are not valid for all kind of tasks. For other types of tasks (supported in Hugging face) you have to refer to their github for more details.<br>

---------------------------------------------------------------------
<i>
{"sentence1":"Seul ouvrage fran\u00e7ais sur le th\u00e8me Produits Structur\u00e9s \/ fonds \u00e0 formule, il permet de fa\u00e7on p\u00e9dagogique d'appr\u00e9hender parfaitement les m\u00e9canismes financiers utilis\u00e9s. Une r\u00e9f\u00e9rence pour ceux qui veulent comprendre les technicit\u00e9s de base et les raisons de l'engouement des investisseurs sur ces actifs \u00e0 hauteur de plusieurs milliards d'euros.","label":"1"}<br>
{"sentence1":"Livre tr\u00e8s int\u00e9ressant !  mais si comme moi vous cherchez des \"infos\" sur les techniques de sorties et autres \"modes d'emploi\", afin de vivre par vous m\u00eame ce genre d'exp\u00e9rience, c'est pas le bon livre.  \u00e7a ne lui enl\u00e8ve d'ailleurd rien \u00e0 son int\u00earet.","label":"0"}
</i>

---------------------------------------------------------------------



In [ ]:
import json

SPLITS=['train', 'test', 'valid']

for split in SPLITS:
    with open('data/cls.books/'+split+'.review', 'r') as f:
        reviews = f.readlines()
    with open('data/cls.books/'+split+'.label', 'r') as f:
        labels = f.readlines()
    with open('data/cls.books-json/'+split+'.json', 'w') as f:   
        #fill the gap here to create train.json, valid.json and test.json


## <b>Finetuning $RoBERTa_{small}^{fr}$ using the Transformers Library</b>

In order to finrtune the model using HuggingFace, you to use the <b>run_glue.py</b> Python script located in the transformers library. For more details, refer to <a href="https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification" target="_blank">the Huggingface/transformers repository on Github</a>. Make sure to use the same hyperparameter as in the first part of this lab.

In [ ]:
DATA_SET='books'
MODEL='RoBERTa_small_fr_huggingface'
MAX_SENTENCES= # fill me, batch size.
LR=#fill me, learning rate
MAX_EPOCH=#fill me
NUM_CLASSES=#fill me
SEEDS=3
CUDA_VISIBLE_DEVICES=0

In [ ]:
for SEED in range(SEEDS):
  SAVE_DIR= 'checkpoints/'+TASK+'/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \
    
  )#fill me 

In [ ]:
%tensorboard --logdir checkpoints